In [22]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,679468,36.3,1292165,69.1,1292165,69.1
Vcells,1283732,9.8,121587244,927.7,189838875,1448.4


In [23]:
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
#setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread("./datasets/comp1/competencia1_2022_fe_inicial.csv")  

In [24]:
#------------------------------------------------------------------------------
#deflaciona por IPC
#momento 1.0  31-dic-2020 a las 23:59

drift_deflacion  <- function( campos_monetarios )
{
  vfoto_mes <- c( 201901, 201902, 201903, 201904, 201905, 201906,
                  201907, 201908, 201909, 201910, 201911, 201912,
                  202001, 202002, 202003, 202004, 202005, 202006,
                  202007, 202008, 202009, 202010, 202011, 202012,
                  202101, 202102, 202103, 202104, 202105 )

  vIPC  <- c( 1.9903030878, 1.9174403544, 1.8296186587,
              1.7728862972, 1.7212488323, 1.6776304408,
              1.6431248196, 1.5814483345, 1.4947526791,
              1.4484037589, 1.3913580777, 1.3404220402,
              1.3154288912, 1.2921698342, 1.2472681797,
              1.2300475145, 1.2118694724, 1.1881073259,
              1.1693969743, 1.1375456949, 1.1065619600,
              1.0681100000, 1.0370000000, 1.0000000000,
              0.9680542110, 0.9344152616, 0.8882274350,
              0.8532444140, 0.8251880213 )

  tb_IPC  <- data.table( "foto_mes"= vfoto_mes,
                         "IPC" = vIPC )

  dataset[ tb_IPC,
           on= c("foto_mes"),
           (campos_monetarios) :=  .SD * i.IPC ,
           .SDcols = campos_monetarios ]

}

In [25]:
#------------------------------------------------------------------------------

drift_rank_simple  <- function( campos_drift )
{
  for( campo in campos_drift )
  {
    cat( campo, " " )
    dataset[ , paste0(campo,"_rank") :=  (frank( get(campo), ties.method="random") - 1) / ( .N -1 ), by= foto_mes]
    dataset[ , (campo) := NULL ]
  }
}


In [26]:
#------------------------------------------------------------------------------
#El cero se transforma en cero
#los positivos se rankean por su lado
#los negativos se rankean por su lado

drift_rank_cero_fijo  <- function( campos_drift )
{
  for( campo in campos_drift )
  {
    cat( campo, " " )
    dataset[ get(campo) ==0, paste0(campo,"_rank") := 0 ]
    dataset[ get(campo) > 0, paste0(campo,"_rank") :=   frank(  get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ get(campo) < 0, paste0(campo,"_rank") :=  -frank( -get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ , (campo) := NULL ]
  }
}


In [27]:
setorder( dataset, foto_mes, numero_de_cliente )

#por como armé los nombres de campos, estos son los campos que expresan variables monetarias
campos_monetarios  <- colnames(dataset)
campos_monetarios  <- campos_monetarios[campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)"]

#aqui aplico un metodo para atacar el data drifting
#hay que probar experimentalmente cual funciona mejor

drift_rank_simple( campos_monetarios )
#drift_rank_cero_fijo( campos_monetarios )
#drift_deflacion( campos_monetarios ) 


mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente_adicional  mcuenta_corriente  mcaja_ahorro  mcaja_ahorro_adicional  mcaja_ahorro_dolares  mcuentas_saldo  mautoservicio  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_personales  mprestamos_prendarios  mprestamos_hipotecarios  mplazo_fijo_dolares  mplazo_fijo_pesos  minversion1_pesos  minversion1_dolares  minversion2  mpayroll  mpayroll2  mcuenta_debitos_automaticos  mttarjeta_visa_debitos_automaticos  mttarjeta_master_debitos_automaticos  mpagodeservicios  mpagomiscuentas  mcajeros_propios_descuentos  mtarjeta_visa_descuentos  mtarjeta_master_descuentos  mcomisiones_mantenimiento  mcomisiones_otras  mforex_buy  mforex_sell  mtransferencias_recibidas  mtransferencias_emitidas  mextraccion_autoservicio  mcheques_depositados  mcheques_emitidos  mcheques_depositados_rechazados  mcheques_emitidos_rechazados  matm  matm_other  Master_mfinanciacion_limite  Master_msaldototal  Ma

In [28]:
fwrite( dataset,
        file="./datasets/comp1/competencia1_2022_fe_inicial_rank0simple.csv",
        sep= "," )